# **Clustering Online Retail Data**

The code below is taken from Manish Kumar's submission on [kaggle.com](https://www.kaggle.com/hellbuoy/online-retail-k-means-hierarchical-clustering/notebook).

You are encouraged to go to the link above and check the full code. In this lab, you will do the necessary steps to explore the data and prepare it for sklearn algorithms.

**About the data set**

Online retail is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

Using K-Means Clustering, you can segement the Customers based on RFM so that the company can target its customers efficiently.

**Import libraries**

In [1]:
# import required libraries for dataframe and visualization

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
import datetime as dt

# import required sklearn libraries for KMeans Clustering
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Acquire data

In [2]:
# Read in the data using panda's read_csv method
retail = pd.read_csv('UnsupervisedLearning/OnlineRetail/OnlineRetail.csv', sep=",", encoding="ISO-8859-1", header=0)

#TODO: Write code to inspect the first five rows of the data frame


# Inspect data

In [3]:
#TODO: Write code to inspect the shape of the data frame


In [4]:
#TODO: Write code to display information about the data frame


In [5]:
#TODO: Write code to display statistics about the data frame


# Clean data

**Correcting**

In [6]:
#TODO: Write code to drop rows having missing values


In [7]:
# Changing the datatype of Customer Id to a string as per Business understanding

retail['CustomerID'] = retail['CustomerID'].astype(str)

**Creating**

We are going to analysis the Customers based on 3 factors:
* R (Recency): Number of days since last purchase
* F (Frequency): Number of transactions
* M (Monetary): Total amount of transactions (revenue contributed)

In [8]:
# Create a new attribute : Monetary based on Quantity and UnitPrice

retail['Amount'] = retail['Quantity']*retail['UnitPrice']
rfm_m = retail.groupby('CustomerID')['Amount'].sum()
rfm_m = rfm_m.reset_index()

#TODO: Write code to inspect the first five rows of rfm_m


In [9]:
# Create a new attribute : Frequency based on invoice count per customer ID

rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequency']

#TODO: Write code to inspect the first five rows of rfm_f


In [10]:
# Merge the two data frames

rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')

#TODO: Write code to inspect the first five rows of rfm

In [11]:
# Create a new attribute : Recency
# -----------------------------------

# Convert to datetime to proper datatype
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%d-%m-%Y %H:%M')

# Compute the maximum date to know the last transaction date
max_date = max(retail['InvoiceDate'])

# Compute the difference between max date and transaction date
retail['Diff'] = max_date - retail['InvoiceDate']

# Compute last transaction date to get the recency of customers
rfm_p = retail.groupby('CustomerID')['Diff'].min()
rfm_p = rfm_p.reset_index()

# Extract number of days only
rfm_p['Diff'] = rfm_p['Diff'].dt.days

# Merge tha dataframes to get the final RFM dataframe
rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')
rfm.columns = ['CustomerID', 'Amount', 'Frequency', 'Recency']

#TODO: Write code to inspect the first five rows of the data frame


**Remove outliers**

In [12]:
# Outlier Analysis of Amount Frequency and Recency

attributes = ['Amount','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rfm[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

In [13]:
# Remove outliers for Amount
Q1 = rfm.Amount.quantile(0.05)
Q3 = rfm.Amount.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Amount >= Q1 - 1.5*IQR) & (rfm.Amount <= Q3 + 1.5*IQR)]

#TODO: Write code to remove outliers for Recency

# Remove outliers for Frequency
Q1 = rfm.Frequency.quantile(0.05)
Q3 = rfm.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequency >= Q1 - 1.5*IQR) & (rfm.Frequency <= Q3 + 1.5*IQR)]

**Rescale attributes**

In [14]:
# Rescaling the attributes using a MinMaxScaler

rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate
scaler = MinMaxScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)

rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']
rfm_df_scaled.head()

# Earn Your Wings

Implement a K-Means Clustering algorithm on the cleaned data set. Use the elbow method to find the right value of k to use.
Add comments in your code to explain each step that you take in your implementation.